In [1]:
!nvidia-smi

Mon Aug 29 15:16:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [2]:
%%capture
!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install sentencepiece

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import re

import random
import torch

def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

import warnings
warnings.simplefilter('ignore')

import gc

from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
drive.mount('/content/drive', force_remount=False)
%cd /content/drive/MyDrive/signate_student_cup_2022

Mounted at /content/drive
/content/drive/MyDrive/signate_student_cup_2022


In [5]:
df = pd.read_csv('data/augmented_data.csv', index_col=0)
df

,description,jobflag,len,temperature,top_p,top_k,repetition_penalty,input_txt
0,ensuring the production of usable scientific d...,1,140,1.0,0.88,80,2.0,The data scientst's duties include
1,the analysis of research materials and relevan...,1,60,1.0,0.88,120,1.5,The data scientst's duties include
2,determining where sufficient time is to make t...,1,140,1.8,0.88,120,1.5,The data scientst's duties include
3,collecting and analysing research findings. In...,1,140,1.4,0.80,120,2.0,The data scientst's duties include
4,identifying how much of your job involves answ...,1,100,1.4,0.84,120,2.0,The data scientst's duties include
...,...,...,...,...,...,...,...,...
243,running quality control programs or technical ...,4,140,1.8,0.84,120,1.5,The consultant's duties include
244,giving the superintendent some guidance about ...,4,100,1.0,0.92,120,2.0,The consultant's duties include
245,managing security access to client networks. O...,4,60,1.8,0.92,120,1.5,The IT consultant's duties include
246,the following:. Assessing current project and ...,4,60,1.0,0.92,80,2.0,The consultant's duties include


In [6]:
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained('facebook/mbart-large-50-one-to-many-mmt')
mbart_translator = pipeline('translation',
                            model='facebook/mbart-large-50-one-to-many-mmt',
                            src_lang='en_XX', tgt_lang='ja_XX', device=0)

for i, dec in enumerate(df["description"]):
    print("==========================")

    print(f'Label={df["jobflag"][i]}')
    print(mbart_translator(dec)[0]["translation_text"])
    print(dec)

Label=1
利用可能な科学データセットの生産を確保し、すべてのレベルの科学者のための教育環境でのサポートを提供する データ科学プログラムは、大学院の間だけでなく、卒業後に恒久的に学生のインターンシップを通じて、学部生からの参加を奨励します。 *. 資格要件: このプログラムに参加するには、2012年9月から2016年4月まで(フルタイム)カナダ内の当社の監督下で直接雇用または以前に働いていた必要があります。 または、2017年8月までに博士号を取得しました。
ensuring the production of usable scientific datasets, and to provide support in an educational environment for scientists at all levels. The Data Sciences Program will encourage participation from undergraduate students throughout their time with us through student internships during grad school as well as on a permanent basis upon completion.*. Eligibility Requirements: To participate in this program you must have been employed or formerly worked directly under our supervision within Canada* between September 2012 – April 2016 (full-time), OR had obtained your PhD by August 2017 AND be enrolled part 1/2 semester after receipt*, if eligible full course load has not yet started.* Please note that only graduates completing studies are included - recent post